In [10]:
import Random
rng = Random.MersenneTwister();

import ForwardDiff as fd
import ReverseDiff as rd

import LinearAlgebra as la

using BenchmarkTools: @btime

include("NeuralNetwork/layer.jl")
include("NeuralNetwork/neuralnetwork.jl")

gradient (generic function with 1 method)

In [23]:
positions = rand(rng, Float64, 5)

5-element Vector{Float64}:
 0.43251896313989735
 0.8765201933852143
 0.42482832345377175
 0.07815120371406215
 0.8904149617608421

## Neural Network

In [50]:
nn = NeuralNetwork([
    Dense((50, 5), rng),
    Dense((30, 50), rng),
    Dense((30, 30), rng),
    Dense((30, 30), rng),
    Dense((30, 30), rng),
    Dense((30, 30), rng),
    Dense((30, 30), rng),
    Dense((30, 30), rng),
    Sigmoid(),
        DeArray()
        ], positions);

In [29]:
@time nn = NeuralNetwork([Dense((5, 5), rng),
    Dense((3, 5), rng),
    Dense((1, 3), rng),
    Exp(),
    DeArray()], positions);

  0.000154 seconds (379 allocations: 24.438 KiB)


In [66]:
@time model(positions, nn);

  0.000022 seconds (19 allocations: 5.734 KiB)


In [52]:
@time model_flat(positions, nn, nn.params_flat);

  0.000039 seconds (92 allocations: 68.594 KiB)


In [63]:
@btime kinetic(positions, nn);

  32.400 μs (994 allocations: 73.70 KiB)


In [62]:
@btime gradient(positions, nn);

  55.900 μs (7 allocations: 116.86 KiB)


In [8]:
grad_result = zeros(length(positions), length(positions));
grad_config = rd.HessianConfig(positions)
grad_tape = rd.HessianTape(positions -> model(positions, nn), positions, grad_config)
grad_tape = rd.compile(grad_tape)

typename(ReverseDiff.CompiledTape)(#9)

In [10]:
@time rd.hessian!(grad_result, grad_tape, positions)

  0.000052 seconds


5×5 Matrix{Float64}:
 248.103  265.442  172.092  169.529  283.394
 265.442  283.993  184.119  181.377  303.199
 172.092  184.119  119.369  117.591  196.571
 169.529  181.377  117.591  115.839  193.643
 283.394  303.199  196.571  193.643  323.704